In [1]:
# !pip install nbimporter   # with this i can import ipynb file in another ipunb file

In [2]:
# import nbimporter

In [3]:
# jupyter nbconvert --to script util.ipynb


In [4]:
# %run util.ipynb

In [5]:
# pip install PyAutoGUI


In [6]:
# pip install pynput

In [1]:
import cv2 # via opencv
import mediapipe as mp   # For hand gesture control 
import utill
import pyautogui # this library is for mouse movement

from pynput.mouse import Button, Controller
mouse = Controller()  # to control the mouse

In [2]:
import random

In [3]:
screen_width, screen_height = pyautogui.size() # To get the screen width and height


mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode = False,
    model_complexity = 1,
    min_detection_confidence = 0.7,
    min_tracking_confidence = 0.7,
    max_num_hands = 1
)


In [4]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mpHands.HandLandmark.INDEX_FINGER_TIP]
    return None

In [5]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * screen_width)
        y = int(index_finger_tip.y * screen_height)
        pyautogui.moveTo(x,y)

In [6]:
def is_left_click(landmarks_lst, thumb_index_dist):
    return (utill.get_angle(landmarks_lst[5], landmarks_lst[6], landmarks_lst[8])<50 and 
           utill.get_angle(landmarks_lst[9], landmarks_lst[10], landmarks_lst[12])>90 and
           thumb_index_dist > 50
           )
"""above code is checking, 1- (Index finger is bend)
                2 --(Middle finger should be straight)
                3 --(Thumb is also straight
                )"""
def is_right_click(landmarks_lst, thumb_index_dist):
    return (utill.get_angle(landmarks_lst[9], landmarks_lst[10], landmarks_lst[12])<50 and 
           utill.get_angle(landmarks_lst[5], landmarks_lst[6], landmarks_lst[8])>90 and
           thumb_index_dist > 50
           )

def is_double_click(landmarks_lst, thumb_index_dist):
    return (utill.get_angle(landmarks_lst[5], landmarks_lst[6], landmarks_lst[8])<50 and 
           utill.get_angle(landmarks_lst[9], landmarks_lst[10], landmarks_lst[12])<50 and
           thumb_index_dist > 50
           )

#Change
def is_screenshot(landmarks_lst, thumb_index_dist):
    return (utill.get_angle(landmarks_lst[5], landmarks_lst[6], landmarks_lst[8])<50 and 
           utill.get_angle(landmarks_lst[9], landmarks_lst[10], landmarks_lst[12])<50 and
           thumb_index_dist < 50
           )


In [7]:
def detect_gesture(frame, landmarks_lst, processed):
    if len (landmarks_lst)>=21:
    # first thing is to move the mouse
        # second to track the point and mark that point
        index_finger_tip = find_finger_tip(processed)
        # print (index_finger_tip)
        # thumb
        thumb_index_dist = utill.get_distance([landmarks_lst[4], landmarks_lst[5]])

        if thumb_index_dist < 50 and utill.get_angle(landmarks_lst[5], landmarks_lst[6],landmarks_lst[8])>90:
            move_mouse(index_finger_tip)


        # Left click -  Thumb - open/strainght, Index - bend, Middle finger - Straight  - use pynpuy
        elif is_left_click(landmarks_lst, thumb_index_dist):
            print ("Left clicked")
            mouse.press(Button.left)
            mouse.release(Button.left)
            cv2.putText(frame, "Left Click",(50,50),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)
        
        # Right click   - use pynput
        elif is_right_click(landmarks_lst, thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)
            cv2.putText(frame, "Right Click",(50,50),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)

        #Double click  - using pyatuogui for this / or can use another (find it and apply)
        elif is_double_click(landmarks_lst, thumb_index_dist):
            pyautogui.doubleClick()
            cv2.putText(frame, "Double Click", (50,50),cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0),2)
            
        #Screen Shot - Thumb/index/ middle finger should be closed
        elif is_screenshot(landmarks_lst, thumb_index_dist):
            im1 = pyautogui.screenshot()   
            label = random.randint(1,1000)  # As it'll take multiple screenshot, to store those it generates random no. as label
            im1.save(f"ScreenShot {label}.png") # To save the img as png extension
            cv2.putText(frame, "Screen Shot Taken", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)

In [8]:
def main():
    cap = cv2.VideoCapture(0)  #0 means inbuild camera / primary camera
    draw = mp.solutions.drawing_utils  # to show the line between the keypoints/ landmarks
    try:
        while cap.isOpened():
            ret, frame = cap.read()   #ret - return is the bool value which returns the frame

            if not ret:
                break
            frame = cv2.flip(frame, 1)   # Flip the frame so it'll look like we're watching the mirror
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            processed = hands.process(frameRGB)  # it'll process the frame and read all the landmarks

            landmarks_lst = []

            if processed.multi_hand_landmarks:
                hand_landmarks =processed.multi_hand_landmarks[0]
                draw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)

                for lm in hand_landmarks.landmark:
                    landmarks_lst.append((lm.x, lm.y))

            # Detect gestures
            detect_gesture(frame, landmarks_lst, processed)

            # print(landmarks_lst)
            
            cv2.imshow('Frame', frame)
            if cv2.waitKey(3) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

C:\Users\Farogh\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 